## Working with tables

### Task I - Table creation
* create table messages 
* take data from questions (question_id, creation_date, body, user_id)
* partition the table by year (derived from creation_date)

### Task II - Table append
* append to the table new data
* take data from answers with the same structure
* partition by year & append to the table messages

### Task III - Partitions overwrite
* overwrite only partition for the year 2018
* take data from questions but filter only for year 2018
* use insertInto with dynamic overwrite

### Task IV - Tables management
* list all tables that we have in our database
* see the properties of the messages table
* rename the table messages -> posts
* see all partitions that the table has
* see properties of the partiton year=2018
* compute and show the statistics for the table posts
* drop the table posts


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year
from pyspark.sql.types import *

import os

In [ ]:
spark = (
    SparkSession
    .builder
    .appName('Tables')
    .enableHiveSupport()
    .getOrCreate()
)

In [ ]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-3]) 

questions_input_path = os.path.join(project_path, 'output/questions-transformed')

answers_input_path = os.path.join(project_path, 'data/answers')

messages_path = os.path.join(project_path, 'output/tables/messages')

In [ ]:
my_schema = StructType([
    StructField('question_id', LongType()),
    StructField('creation_date', TimestampType()),
    StructField('body', StringType()),
    StructField('user_id', LongType())
])

In [ ]:
questionsDF = spark.read.schema(my_schema).parquet(questions_input_path)

In [ ]:
answersDF = spark.read.schema(my_schema).parquet(answers_input_path)

### Task I

* create partitioned table `messages`, partition it by `year`
* use [year](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.year.html#pyspark.sql.functions.year) to derive the partition column from `creation_date`
* save it at `messages_path`
* use [write](http://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.write.html#pyspark.sql.DataFrame.write) and [saveAsTable](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameWriter.saveAsTable.html#pyspark.sql.DataFrameWriter.saveAsTable)

In [ ]:
# your code here:

### Task II

* append the table `messages` with new data
* the new data is the `answersDF`

In [ ]:
spark.table("messages").count()

In [ ]:
# your code here:


In [ ]:
answersDF.count()

In [ ]:
spark.table("messages").count() # test if the count increased with the append

### Task III

* ovewrite single partition in the table `messages`
* take data from questionsDF for the year=2018 and use this data to overwrite 2018 partiton in `messages`
* see [insertInto](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameWriter.insertInto.html#pyspark.sql.DataFrameWriter.insertInto) functionality with different partitionOverwriteModes
 * STATIC is default (this can overwrite the whole dataset)
 * DYNAMIC will overwrite only relevant partition

In [ ]:
spark.conf.get("spark.sql.sources.partitionOverwriteMode")

In [ ]:
# set the conf to DYNAMIC:


In [ ]:
# see how many rows we have in questionsDF for 2018
(
    questionsDF
    .withColumn("year", year("creation_date"))
    .filter(col('year') == 2018)
).count()

In [ ]:
(
    spark.table('messages')
    .filter(col('year') == 2018)
).count()

In [ ]:
# Let's first see the overwrite=False option which appends the partition:


In [ ]:
# test the count:

(
    spark.table('messages')
    .filter(col('year') == 2018)
).count()

In [ ]:
# overwrite=True option which overwrites the partition:


In [ ]:
spark.table('messages').filter(col('year') == 2018).count() # this partition was overwritten

In [ ]:
spark.table('messages').filter(col('year') == 2019).count() # this partition didn't change

#### Task IIIb

Let' see what happens in the default STATIC mode

In [ ]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "STATIC")

In [ ]:
# STATIC with overwrite=False appends the partition


In [ ]:
spark.table('messages').filter(col('year') == 2018).count() # this partition was appended

In [ ]:
# STATIC with overwrite=TRUE overwrites the entire table:


In [ ]:
spark.table('messages').filter(col('year') == 2018).count() # this partition was overwritten

In [ ]:
spark.table('messages').filter(col('year') == 2019).count() # all other partitions were deleted

### Task IV - Tables management
* list all tables that we have in our database
* see the properties of the messages table
* rename the table messages -> posts
* see all partitions that the table has
* see properties of the partiton year=2018

Hint:
* check the sql-reference [docs](https://spark.apache.org/docs/latest/sql-ref.html)
* check [catalog API](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Catalog.html#pyspark.sql.Catalog)

In [ ]:
# list all tables:


In [ ]:
# See properties of a table


In [ ]:
# Change the name of the table to posts:


In [ ]:
# See partitions of the table


In [ ]:
# See properties of a single partition:


In [ ]:
# Compute the statistics

In [ ]:
# Show the computed statistics

In [ ]:
# Drop the table posts:


To see more information about saving data with Spark, feel free to check my [article](https://towardsdatascience.com/notes-about-saving-data-with-spark-3-0-86ba85ca2b71).

In [ ]:
spark.stop()